In [1]:
# Cài thêm các thư viện cần thiết
!pip install -U transformers accelerate bitsandbytes deep-translator -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 76.0 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 24.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.0 MB/s eta 0:00:00:00:0100:01


In [2]:
# Import cần thiết
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
from deep_translator import GoogleTranslator

2025-06-30 09:29:03.002447: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751275743.221486      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751275743.278818      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Hàm dịch tiếng Việt → tiếng Anh
def translate_vi_to_en(text):
    return GoogleTranslator(source='vi', target='en').translate(text)

In [4]:
# Config để load mô hình 4bit (tiết kiệm VRAM)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,   # hoặc bfloat16 nếu GPU hỗ trợ
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# Tên mô hình từ HuggingFace
model_id = "hahuy2004/mistral-metamathqa-lora-365k"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load model đã quant hóa 4bit + tự động chia lên GPU phù hợp
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
)

# Khởi tạo pipeline inference (dễ dùng, nhưng giới hạn nhỏ hơn)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/110 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/812 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:222: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

Device set to use cuda:0


In [7]:
# Câu hỏi tiếng Việt
# question_vi = "Chuyển số 10101₃ sang số nguyên hệ 10. Giải thích từng bước và đưa ra kết quả cuối cùng."
# question_en = translate_vi_to_en("Convert $10101_3$ to a base 10 integer.")
# question_en = "Randy, Peter, and Quincy all drew pictures. Peter drew 8 pictures. Quincy drew 20 more pictures than Peter. If they drew 41 pictures altogether, how many did Randy draw?"
question_en = "There are 6 people going on an airplane trip. They each have 5 bags of luggage. Each of their bags weighs the maximum weight allowed, 50 pounds. The airplane can hold a total luggage weight of 6000 pounds. How many more bags at maximum weight can the plane hold?"
print(question_en)

# # Câu prompt
# translated_prompt = f"""
# Question: {question_en}
# Answer and explain your reasoning step by step. Include all powers of 3 used and the final result.
# """
translated_prompt = f"""
You are a helpful and precise mathematical reasoning assistant.

Given a question, your task is to solve it correctly and explain your reasoning step by step. Include any relevant formulas, intermediate steps, and the final answer. If the question includes multiple-choice options, clearly indicate the correct one and justify your choice.

Question: {question_en}

Answer and explain your reasoning step by step:
"""

There are 6 people going on an airplane trip. They each have 5 bags of luggage. Each of their bags weighs the maximum weight allowed, 50 pounds. The airplane can hold a total luggage weight of 6000 pounds. How many more bags at maximum weight can the plane hold?


In [8]:
# Test model
output_en = pipe(translated_prompt, max_new_tokens=512, do_sample=False)[0]["generated_text"]
# print(output)

# Cắt phần prompt đầu (nếu cần, tùy cách dùng)
response_en = output_en[len(translated_prompt):].strip()

# Dịch kết quả sang tiếng Việt
response_vi = GoogleTranslator(source='en', target='vi').translate(response_en)

# In cả hai kết quả
print("🔤 English Response:\n", response_en)
print("\n🇻🇳 Dịch tiếng Việt:\n", response_vi)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🔤 English Response:
 Each person has 5 bags of luggage, and each bag weighs 50 pounds, so each person's total luggage weight is 5 * 50 = 250 pounds.

There are 6 people going on the trip, so the total luggage weight for all 6 people is 6 * 250 = 1500 pounds.

The airplane can hold a total luggage weight of 6000 pounds, so it can hold 6000 - 1500 = 4500 pounds more.

Therefore, the plane can hold 4500 / 50 = 90 more bags at maximum weight.

The answer is: 90

🇻🇳 Dịch tiếng Việt:
 Mỗi người có 5 túi hành lý và mỗi túi nặng 50 pound, vì vậy tổng trọng lượng hành lý của mỗi người là 5 * 50 = 250 pounds.

Có 6 người đang đi trong chuyến đi, vì vậy tổng trọng lượng hành lý cho cả 6 người là 6 * 250 = 1500 pounds.

Máy bay có thể chứa tổng trọng lượng hành lý là 6000 pounds, do đó nó có thể giữ 6000 - 1500 = 4500 pounds hơn.

Do đó, mặt phẳng có thể chứa 4500 /50 = 90 túi ở trọng lượng tối đa.

Câu trả lời là: 90
